In [1]:
import numpy as np
import pybullet as p
import pandas as pd
import time
from robot_descriptions.loaders.pybullet import load_robot_description

pybullet build time: Nov 29 2024 16:26:49


In [2]:
p.connect(p.GUI)
p.setTimeStep(0.0001)
p.setGravity(0, 0, -9.81)

Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [4]:
from robot_descriptions import ur5_description
print(ur5_description.URDF_PATH)

/Users/theo/.cache/robot_descriptions/example-robot-data/robots/ur_description/urdf/ur5_robot.urdf


In [3]:
robot_id = load_robot_description('ur5_description', useFixedBase=True) 
joint_indices = [i for i in range(p.getNumJoints(robot_id)) if p.getJointInfo(robot_id, i)[2] == p.JOINT_REVOLUTE]
for i in range(p.getNumJoints(robot_id)):
  print(p.getJointInfo(robot_id, i))
  print(p.getJointInfo(robot_id, i)[2] == p.JOINT_REVOLUTE)

immovable_joints = [6, 8]
wrist3_joint = 6
eff_idx = 7
pendulum_joint_idx = 8
print("End effector joint: ", eff_idx)
print("Revolute joints: ", joint_indices)

(0, b'world_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'base_link', (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), -1)
False
(1, b'shoulder_pan_joint', 0, 7, 6, 1, 0.0, 0.0, -6.28318530718, 6.28318530718, 150.0, 3.15, b'shoulder_link', (0.0, 0.0, 1.0), (0.0, 0.0, 0.089159), (0.0, 0.0, 0.0, 1.0), 0)
True
(2, b'shoulder_lift_joint', 0, 8, 7, 1, 0.0, 0.0, -6.28318530718, 6.28318530718, 150.0, 3.15, b'upper_arm_link', (0.0, 1.0, 0.0), (0.0, 0.13585, 0.0), (0.0, -0.7071067811848163, 0.0, 0.7071067811882787), 1)
True
(3, b'elbow_joint', 0, 9, 8, 1, 0.0, 0.0, -3.14159265359, 3.14159265359, 150.0, 3.15, b'forearm_link', (0.0, 1.0, 0.0), (0.0, -0.1197, 0.14499999999999996), (0.0, 0.0, 0.0, 1.0), 2)
True
(4, b'wrist_1_joint', 0, 10, 9, 1, 0.0, 0.0, -6.28318530718, 6.28318530718, 28.0, 3.2, b'wrist_1_link', (0.0, 1.0, 0.0), (0.0, 0.0, 0.14225), (0.0, -0.7071067811848163, 0.0, 0.7071067811882787), 3)
True
(5, b'wrist_2_joint', 0, 11, 10, 1, 0.0, 0.0, -6.28318530718, 6.283185307

# Inverse kinematics with preferred positions

In [4]:
#lower limits for null space
ll = [3.14 / 18 * x for x in [-36, 3.5, -6.5, 1.5, -4, 17.5, 0]]
#upper limits for null space
ul = [3.14 / 18 * x for x in [36,  4.5, -5.5, 4.5,  4, 17.5, 0]]
#joint ranges for null space
jr = [3.14 / 18  * x for x in [72,    1,    1,   3,  8,    0, 0]]
#restposes for null space
rp = [3.14 / 18 * x for x in [0,     4,   -6,   3,  0, 17.5, 0]]

print(ll)
print(ul)
print(jr)
print(rp)

def smoothInverseKinematics(robot_id, eff_idx, target_pos):
  current_positon = p.getLinkState(robot_id, eff_idx)[0]
  return p.calculateInverseKinematics(
    robot_id,
    eff_idx,
    target_pos,
    lowerLimits = ll,
    upperLimits = ul,
    jointRanges = jr,
    restPoses   = rp,
    solver=420,
    maxNumIterations=20,
    residualThreshold=.01
  )

[-6.28, 0.6105555555555556, -1.133888888888889, 0.2616666666666667, -0.6977777777777778, 3.052777777777778, 0.0]
[6.28, 0.785, -0.9594444444444445, 0.785, 0.6977777777777778, 3.052777777777778, 0.0]
[12.56, 0.17444444444444446, 0.17444444444444446, 0.5233333333333334, 1.3955555555555557, 0.0, 0.0]
[0.0, 0.6977777777777778, -1.0466666666666669, 0.5233333333333334, 0.0, 3.052777777777778, 0.0]


# Disable default motor control

In [5]:
for i in joint_indices:
    p.setJointMotorControl2(
        bodyUniqueId=robot_id,
        jointIndex=i,
        controlMode=p.VELOCITY_CONTROL,
        force=0  # Disable motor forces
    )

# Feedback and Stabilization

In [6]:
def pd_control(robot_id, q_desired, v_desired, Kp, Kd):
  joint_states = p.getJointStates(robot_id, joint_indices)
  q_measured = np.array([state[0] for state in joint_states])
  v_measured = np.array([state[1] for state in joint_states])
  position_error = q_desired - q_measured
  velocity_error = v_desired - v_measured
  
  tau = Kp * position_error + Kd * velocity_error
  # add gravity compensation
  tau += np.array(p.calculateInverseDynamics(robot_id, q_measured.tolist(), [0] * len(joint_indices), [0] * len(joint_indices)))
  for i in range(len(joint_indices)):
    if joint_indices[i] in immovable_joints: # ensure pendulum and wrist 3 joint is commanded with zero torque
      tau[i] = 0
  return tau

Kp = 100 * np.ones_like(joint_indices)
Kd = 5 * np.ones_like(joint_indices)

def control_to_desired_configuration(robot_id, q_desired, v_desired=None, max_steps=5000, sleep=None):
  if v_desired is None:
    v_desired = np.zeros_like(q_desired)
  for step in range(max_steps):  # Simulation steps
    torques = pd_control(robot_id, q_desired, v_desired, Kp, Kd).tolist()
    p.setJointMotorControlArray(
      bodyUniqueId=robot_id,
      jointIndices=joint_indices,
      controlMode=p.TORQUE_CONTROL,
      forces=torques
    )
    p.stepSimulation()
    if sleep is not None:
      time.sleep(sleep)
    

# Resetting to balanced pose

In [17]:
rest_pose_balancing = [np.pi / 18 * x for x in [0, 4, -6, 3, 0, 17.15, 0]]
def reset_to_balancing_pose(robot_id):
  for i in range(len(joint_indices)): 
    p.resetJointState(robot_id, joint_indices[i], targetValue=-rest_pose_balancing[i], targetVelocity=0)
    p.setJointMotorControl2(
          bodyUniqueId=robot_id,
          jointIndex=joint_indices[i],
          controlMode=p.VELOCITY_CONTROL,
          force=0  # Disable motor forces
      )

In [20]:
rest_pose_swing = [np.pi / 18 * x for x in [0, 4, -6, 3, 0, 17.15, 18]]
def reset_to_swing_pose(robot_id):
  for i in range(len(joint_indices)): 
    p.resetJointState(robot_id, joint_indices[i], targetValue=-rest_pose_swing[i], targetVelocity=0)
    p.setJointMotorControl2(
          bodyUniqueId=robot_id,
          jointIndex=joint_indices[i],
          controlMode=p.VELOCITY_CONTROL,
          force=0  # Disable motor forces
      )

In [18]:
reset_to_balancing_pose(robot_id)

In [19]:
print(p.getLinkState(robot_id, eff_idx))

((0.7105991188452455, 0.19145000000000006, 0.13497427907824033), (-0.009255741794926584, -0.009255741794971906, 0.7070462016313909, 0.7070462016348529), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0), (0.7105991244316101, 0.19144999980926514, 0.13497428596019745), (-0.00925574079155922, -0.00925574079155922, 0.7070462107658386, 0.7070462107658386))


In [11]:
p.calculateInverseKinematics(robot_id, eff_idx, (0.7105991188452455, 0.19145000000000006, 0.13497427907824033))

(3.4187670006900682e-18,
 -0.6981317007977319,
 1.0471975511965974,
 -0.5235987755982988,
 3.2627810165066864e-17,
 -2.993239667170275,
 0.0)

# Moving end-effector along a horizontal line

## Forward movement

In [12]:
# Define the start point and direction of the line
reset_to_balancing_pose(robot_id)
start_point =p.getLinkState(robot_id, eff_idx)[0] #np.array([0.7105991188452455, 0.19145000000000006, 0.13497427907824033])  # [x, y, z] in 3D space
direction_vector = np.array([+0.3, 0, 0])  # Direction along X-axis (horizontal)
target_positions = start_point + np.linspace(0, 1, num=10)[:, np.newaxis] * direction_vector[np.newaxis, :]
print(target_positions[0], target_positions[-1])
print(p.getJointInfo(robot_id, eff_idx))

time.sleep(1)
for target in target_positions:
  joint_angles = p.calculateInverseKinematics(robot_id,eff_idx,target)
  control_to_desired_configuration(robot_id, joint_angles)
  time.sleep(1 / 1000.)

[0.71059912 0.19145    0.13497428] [1.01059912 0.19145    0.13497428]
(7, b'ee_fixed_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'ee_link', (0.0, 0.0, 0.0), (0.0, 0.0823, 0.0), (0.0, 0.0, -0.7071067811848163, 0.7071067811882787), 6)


NameError: name 'control_to_desired_configuration' is not defined

## Backward movement

In [22]:
# Define the start point and direction of the line
reset_to_balancing_pose(robot_id)
start_point = p.getLinkState(robot_id, eff_idx)[0] #np.array([0.7105991188452455, 0.19145000000000006, 0.13497427907824033])  # [x, y, z] in 3D space
direction_vector = np.array([-0.3, 0, 0])  # Direction along X-axis (horizontal)
target_positions = start_point + np.linspace(0, 1, num=10)[:, np.newaxis] * direction_vector[np.newaxis, :]
print(target_positions[0], target_positions[-1])
print(p.getJointInfo(robot_id, eff_idx))

time.sleep(1)
for target in target_positions:
  joint_angles = p.calculateInverseKinematics(robot_id,eff_idx,target)
  control_to_desired_configuration(robot_id, joint_angles)
  time.sleep(1 / 1000.)

[0.71059912 0.19145    0.13497428] [0.41059912 0.19145    0.13497428]
(7, b'ee_fixed_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'ee_link', (0.0, 0.0, 0.0), (0.0, 0.0823, 0.0), (0.0, 0.0, -0.7071067811848163, 0.7071067811882787), 6)


In [12]:
reset_to_balancing_pose(robot_id)
start_point = p.getLinkState(robot_id, eff_idx)[0]
left_point = start_point  - np.array([0.3, 0, 0])
right_point = start_point + np.array([0.3, 0, 0])

time.sleep(2)
for i in range(10):
  if i % 2 == 0:
    target = left_point
  else:
    target = right_point
  joint_angles = p.calculateInverseKinematics(robot_id,eff_idx,target)
  control_to_desired_configuration(robot_id, joint_angles)
  time.sleep(0.01)

# Balancing the pendulum

In [27]:
from tqdm import tqdm

class EnvRealistic:
  axis_vector = np.array([0.1, 0, 0])
  action_space = "Continuous [0, 1]"
  def __init__(self, TIME_HORIZON=1000, sleep=None,swing = False ):
    self.TIME_HORIZON = TIME_HORIZON
    self.sleep=sleep
    self.swing= swing
    self.reset()
    

  def rand_action(self):
    return np.random.rand() * 2 - 1

  def convert_3d_coords_to_line_position(self, world_pos):
    return world_pos[0] - self.start_point[0]

  def convert_3d_velo_to_line_velo(self, world_vel):
    return np.dot(EnvRealistic.axis_vector, world_vel)

  def get_state(self):
    (world_pos, _, _, _, _, _, world_velo, _) = p.getLinkState(robot_id, eff_idx, computeLinkVelocity=True)
    x = self.convert_3d_coords_to_line_position(np.array(world_pos))
    x_dot = self.convert_3d_velo_to_line_velo(np.array(world_velo))
    theta = p.getJointState(robot_id, wrist3_joint)[0] - self.initial_theta
    theta_dot = p.getJointState(robot_id, wrist3_joint)[1]
    print('z = ' + str( world_pos[2]))
    return (x, x_dot, theta, theta_dot)

  def reset(self):
    """ 
    Returns state
    """
    self.timestep = 0
    reset_to_balancing_pose(robot_id)
    if self.swing :
      reset_to_swing_pose(robot_id)
    self.initial_theta = p.getJointState(robot_id, wrist3_joint)[0]
    self.start_point = p.getLinkState(robot_id, eff_idx)[0]
    self.left_point = self.start_point - 3 * EnvRealistic.axis_vector
    self.right_point = self.start_point  + 3 * EnvRealistic.axis_vector
    return (0, 0, 0, 0)
  
  def step(self, action):
    """
    Returns tuple (state, dead, truncated)
    """
    if action < -1 or action > 1:
      raise Exception("Action not in range [-1, 1]")
    current_position = p.getLinkState(robot_id, eff_idx)[0]
    target = self.start_point - action * EnvRealistic.axis_vector
    
      
    joint_angles = p.calculateInverseKinematics(robot_id,eff_idx,target)
    control_to_desired_configuration(robot_id, joint_angles, max_steps=100, sleep=self.sleep)
    (x, x_dot, theta, theta_dot) = self.get_state()
    self.timestep += 1
    dead = False
    truncated = False
    #if np.abs(theta) > 0.25:
      #print(theta, " theta DEAD")
      #dead = True
    if x + self.start_point[0] < self.left_point[0] or x + self.start_point[0] > self.right_point[0]:
      print(x, " x DEAD")
      dead = True
    #if self.timestep == self.TIME_HORIZON:
    #  truncated = True
    print(x)
    print(theta)
    return ((x, x_dot, theta, theta_dot), dead, truncated)


def generate_sample_of_system_dynamics_realistic(env, nb_samples):
  samples = []
  s = env.reset()
  for _ in tqdm(range(nb_samples)):
    a = env.rand_action()
    next_s, dead, trunc = env.step(a)
    samples.append({
      'x': s[0], 
      'x_dot': s[1], 
      'theta': s[2], 
      'theta_dot': s[3], 
      'u': a, # mapping inputs to {-1, 1} rather than {0, 1} is important, because otherwise the linear regression needs a non-zero constant to fit well
      'evolution_x': next_s[0], 
      'evolution_x_dot': next_s[1], 
      'evolution_theta': next_s[2],
      'evolution_theta_dot': next_s[3]})
    if dead or trunc:
      s = env.reset()
    else:
      s = next_s
  return pd.DataFrame(samples)

In [15]:
env = EnvRealistic(sleep=0.0001)
samples = generate_sample_of_system_dynamics_realistic(env, 1000)

  5%|▍         | 49/1000 [00:22<08:37,  1.84it/s]

0.2526718783559372  theta DEAD


 13%|█▎        | 127/1000 [00:58<06:36,  2.20it/s]

-0.25994190084610436  theta DEAD


 18%|█▊        | 178/1000 [01:23<07:06,  1.93it/s]

...

 91%|█████████ | 910/1000 [07:15<00:56,  1.60it/s]

0.26140928174644396  theta DEAD


 97%|█████████▋| 971/1000 [07:45<00:14,  2.02it/s]

-0.2547084402446682  theta DEAD


100%|██████████| 1000/1000 [08:02<00:00,  2.07it/s]


In [16]:
print(samples.describe())

                 x        x_dot        theta    theta_dot            u  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean     -0.001909     0.000060    -0.017187    -0.154552     0.026201   
std       0.015746     0.034831     0.089092     0.717232     0.578289   
min      -0.059651    -0.082887    -0.247988    -2.919447    -0.998930   
25%      -0.012710    -0.025985    -0.051297    -0.602443    -0.466904   
50%      -0.000651    -0.000757    -0.005119    -0.143623     0.037517   
75%       0.008667     0.027061     0.010750     0.286480     0.552838   
max       0.037072     0.109760     0.247682     2.248499     0.999485   

       evolution_x  evolution_x_dot  evolution_theta  evolution_theta_dot  
count  1000.000000      1000.000000      1000.000000          1000.000000  
mean     -0.001904        -0.000065        -0.018788            -0.166567  
std       0.015908         0.035122         0.094630             0.755290  
min      -0.059651        -0.

## Fit a Linear regression on the data

In [17]:
from statsmodels.api import OLS

small_angle_samples = samples[np.abs(samples["theta"]) < 0.2]
predictors = small_angle_samples[['x', 'x_dot', 'theta', 'theta_dot', 'u']]
outcomes = ['evolution_x_dot', 'evolution_theta_dot', 'evolution_x', 'evolution_theta']
params = {}
for o in outcomes:
  model = OLS(small_angle_samples[o], predictors)
  fit = model.fit()
  print(f"{o} linear model:")
  print(fit.params)
  print(f"Mean squared error of {o} fit", fit.mse_resid)
  params[o] = fit.params.to_numpy()

evolution_x_dot linear model:
x           -0.738924
x_dot        0.646575
theta        0.152292
theta_dot   -0.024031
u           -0.054746
dtype: float64
Mean squared error of evolution_x_dot fit 5.9855275370732344e-06
evolution_theta_dot linear model:
x           -14.668467
x_dot        -5.007803
theta         5.523742
theta_dot     0.185968
u            -0.709063
dtype: float64
Mean squared error of evolution_theta_dot fit 0.007843903487250408
evolution_x linear model:
x            0.936163
x_dot        0.073373
theta        0.015479
theta_dot   -0.002444
u           -0.004445
dtype: float64
Mean squared error of evolution_x fit 6.091323919145669e-08
evolution_theta linear model:
x           -0.136654
x_dot       -0.045864
theta        1.046681
theta_dot    0.002926
u           -0.007453
dtype: float64
Mean squared error of evolution_theta fit 5.362855924368433e-07


## Computing a state-feedback controller according to a discrete-time linear quadratic regulator design

In [18]:
from control import dlqr

dT = 0.0001
A = np.array([
  params['evolution_x'][:4], #x
  params['evolution_x_dot'][:4], #x_dot
  params['evolution_theta'][:4], #theta
  params['evolution_theta_dot'][:4] #theta_dot
])
B = np.array([
  [params['evolution_x'][4]], #x
  [params['evolution_x_dot'][4]], #x_dot
  [params['evolution_theta'][4]], #theta
  [params['evolution_theta_dot'][4]] #theta_dot
])
Q = np.diag([125, 50, 1200, 25])
R = np.diag([1.5])

K, S, E = dlqr(A, B, Q, R)
print(K)

[[-18.13404124 -13.39970578  27.17428003   0.52563169]]


# Trying out the learned policy

In [19]:
s = env.reset()
time.sleep(2)
while True:
  a = np.clip(np.dot(-K, s), -1, 1)
  print(a)
  s, dead, trunc = env.step(a)
  if dead or trunc:
    print(f"Successfull balancing for {env.timestep} timesteps")
    break

[0.]
[0.00610004]
[0.01309409]
[0.02074829]
[0.02900563]
[0.03775396]
[0.04690327]
[0.05638269]
[0.06612874]
[0.07607949]
[0.08617133]
[0.09633754]
[0.10650818]
[0.11661085]
[0.1265718]
[0.13631735]
[0.14577523]
[0.15487584]
[0.16355335]
[0.17174658]
[0.17939966]
[0.18646254]
[0.19289136]
[0.19864858]
[0.2037032]
[0.20803075]
[0.21161331]
[0.21443951]
[0.21650448]
[0.21780982]
[0.21836354]
[0.21818004]
[0.21727999]
[0.21576628]
[0.2136378]
[0.21057332]
[0.2071383]
[0.20318606]
[0.19876382]
[0.19393746]
[0.18877241]
[0.18333876]
[0.17771067]
[0.17196587]
[0.16618497]
[0.16045082]
[0.15484773]
[0.14946061]
[0.14437411]
[0.1396716]
[0.13543409]
[0.13173916]
[0.12865973]
[0.12626288]
[0.1246086]
[0.12374861]
[0.12372517]
[0.12456999]
[0.1263033]
[0.12893308]
[0.13231932]
[0.13690622]
[0.1421344]
[0.14815361]
[0.15492077]
[0.16236204]
[0.17039886]
[0.1789435]
[0.1879007]
[0.1971695]
[0.20664509]
[0.2162206]
[0.22578884]
[0.23524396]
[0.24448285]
[0.25340634]
[0.26192027]
[0.26993619]
[0.277

In [20]:
actions = [-1, 1] * 50 + [-1] * 20 + [1] * 40 + [-1] * 40 + [1] * 70
env.reset()
time.sleep(2)
for a in actions:
  env.step(a)
  time.sleep(0.00001)

-0.25580320227736486  theta DEAD
-0.2661211785738211  theta DEAD
-0.29191854627585556  theta DEAD
-0.30440954843008905  theta DEAD
-0.3326146564827406  theta DEAD
-0.34752111185843226  theta DEAD
-0.37844157424657476  theta DEAD
-0.3960256866380738  theta DEAD
-0.4300049070387417  theta DEAD
-0.4505476315086425  theta DEAD
-0.48796712553877564  theta DEAD
-0.5117660049801298  theta DEAD
-0.5530469157413873  theta DEAD
-0.5804123742911749  theta DEAD
-0.6260155777668044  theta DEAD
-0.6572651840070152  theta DEAD
-0.707689137322506  theta DEAD
-0.7431392785559949  theta DEAD
-0.798914482463887  theta DEAD
-0.8388688494176666  theta DEAD
-0.900547518411468  theta DEAD
-0.9452818154318323  theta DEAD
-1.0134211606797696  theta DEAD
-1.063163569777481  theta DEAD
-1.1383011394627078  theta DEAD
-1.1932083377149367  theta DEAD
-1.2758282958886622  theta DEAD
-1.3359573530214575  theta DEAD
-1.4264475424260001  theta DEAD
-1.4917249664135892  theta DEAD
-1.5903260685764402  theta DEAD
-1.660

# Swing-up

In [29]:
import pyautogui
from pynput.mouse import Listener

# Function to map cursor x position to action
def cursor_to_action(x, screen_width, action_min=-1, action_max=1):
    return-(( x / screen_width) * (action_max - action_min) + action_min)

# Initialize environment
env = EnvRealistic(TIME_HORIZON=1000,swing = True)

# Main loop to continuously track the cursor position and control the robot
screen_width, screen_height = pyautogui.size()  # Get screen dimensions

# Global flag to stop the loop when mouse is clicked
stop_loop = False

xpos_hist = []
theta_hist = []
action_hist = []

# Listener function to detect mouse click
def on_click(x, y, button, pressed):
    global stop_loop
    if pressed:  # If the mouse button is pressed
        stop_loop = True  # Set the flag to True
# Start the mouse listener in the background
listener = Listener(on_click=on_click)
listener.start()

while not stop_loop :
    # Get the current cursor position
    cursor_x, cursor_y = pyautogui.position()
    
    # Map the cursor's x position to robot action
    action = cursor_to_action(cursor_x, screen_width)
    
    # Step the environment with the new action
    state, dead, truncated = env.step(action)
    xpos_hist.append(state[0])
    theta_hist.append(state[2])
    action_hist.append(action)
    # Optionally, print or log the state or action if needed
    print(f"Cursor X: {cursor_x}, Action: {action}")
    
    # Add a sleep interval to control the loop speed if needed
    time.sleep(0.05)  # Adjust for responsiveness (lower values = faster control)
# Stop the listener once the loop is stopped
listener.stop()

print("Loop stopped by mouse click.")

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


z = 0.13530153514491342
0.00130895849922541
0.011416495230390566
Cursor X: 927, Action: -0.2875000000000001
z = 0.1357603522188403
0.002499487891633345
0.02052730162497829
Cursor X: 816, Action: -0.1333333333333333
z = 0.1362705528058735
0.003737477853807958
0.029701731171705603
Cursor X: 816, Action: -0.1333333333333333
z = 0.13680788555285334
0.005006591340031896
0.038959753984161516
Cursor X: 816, Action: -0.1333333333333333
z = 0.13734682589546404
0.006262914815308385
0.048003930460381206
Cursor X: 816, Action: -0.1333333333333333
z = 0.13786612613352
0.007471904538770779
0.056602673553682425
Cursor X: 816, Action: -0.1333333333333333
z = 0.13834866262414586
0.008607266429257798
0.06458030225977174
Cursor X: 816, Action: -0.1333333333333333
z = 0.1387812652873248
0.009650208500284974
0.07181186026106712
Cursor X: 816, Action: -0.1333333333333333
z = 0.13884445828456018
0.009513846962039274
0.06888967288113124
Cursor X: 642, Action: 0.10833333333333328
z = 0.13816379284439112
0.0072

In [ ]:
#if successful you can get history of xposition,theta and actions
print(xpos_hist)
print(theta_hist)
print(action_hist)

In [ ]:
from scipy.optimize import minimize

# Define the system dynamics
def dynamics(x, u):
    # Example: simple linear dynamics
    A = np.eye(len(x))  # Identity matrix
    B = np.eye(len(u))  # Control influence matrix
    return A @ x + B @ u

# Define the cost function
def cost_function(U, X_d, x0, T):
    U = U.reshape((-1, len(x0)))  # Reshape control vector
    x = x0
    cost = 0
    for k in range(T):
        u = U[k]
        x = dynamics(x, u)  # System dynamics
        cost += np.sum((x - X_d[k])**2) + np.sum(u**2)  # Cost terms
    return cost

# Define problem parameters
T = 100  # Number of timesteps
x_dim = 2  # State dimension
u_dim = 2  # Control dimension
X_d = np.random.rand(T, x_dim)  # Example demonstration trajectory
x0 = np.zeros(x_dim)  # Initial state
U_init = np.zeros((T, u_dim))  # Initial guess for control

# Solve the optimization problem
result = minimize(
    fun=cost_function, 
    x0=U_init.flatten(), 
    args=(X_d, x0, T), 
    method='L-BFGS-B'
)

# Optimal controls
U_optimal = result.x.reshape((T, u_dim))